In [2]:
!pip install requests

In [3]:
import requests
import json

Configurações para Executar Modelos em Docker Local 

In [15]:
#IP do servidor local da aplicação no docker:
app_ip = "localhost"

#Porta do servidor de predição de Fraude especificada no docker:
fraude_porta = "10005"

#Porta do servidor de predição de Inadimplencia especificada no docker:
inadimplencia_porta = "10006"

## OBS: NECESSÁRIO TER O DOCKER INSTALADO NO PC!!! 

## Parte 1 – Propensão de inadimplência (Modelo 'Fake')

In [5]:
# informe o cpf
cpf01 = "123456789"

In [7]:
url01 = "http://{}:{}/inadimplencia?cpf={}".format(app_ip, inadimplencia_porta,cpf01)
print(url01)

response01 = requests.request("POST",url01)
respostas01 = json.loads(response01.text.encode('utf8').decode())
respostas01

http://20.83.189.234:10005/fraude?cpf=123456789


## Parte 2 – Propensão a Fraude (Modelo 'Fake')

In [35]:
# informe o cpf
cpf02 = "45948946241"

In [36]:
url02 = "http://{}:{}/inadimplencia?cpf={}".format(app_ip,inadimplencia_porta,cpf02)
response02 = requests.request("POST",url02)
respostas02 = json.loads(response02.text.encode('utf8').decode())
respostas02

{'Consulta Risco de Inadimplencia para o CPF': '45948946241',
 'Resultado': 1,
 'Situacao': 'inadimplente',
 'Mensagem': 'Obrigado pela chamada de API'}

## Parte 3 – Criação da Plataforma Cognitiva (Model Manager) na Azure.

In [21]:
#IP do Maquina Virtual na Azure:
server_ip = "20.83.189.234"

#Porta do servidor de predição na Azure:
server_port = "443"

url = "http://{}:{}/predict".format(server_ip,server_port)
print(url)

http://20.83.189.234:443/predict


CPF do cliente será submetido aos dois modelos para a análise de Risco de Fraude e Inadimplencia.

In [22]:
# informe o cpf
cpf = "123456789"

In [23]:
response001 = requests.request("POST", url + f"?model=modelo01&cpf={cpf}" )
resposta001 = json.loads(response001.text.encode('utf8').decode())
resposta001

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [19]:
response002 = requests.request("POST", url + f"?model=modelo02&cpf={cpf}" )
resposta002 = json.loads(response002.text.encode('utf8').decode())
resposta002

JSONDecodeError: Expecting value: line 1 column 1 (char 0)